### Version 2 - post submission: ignore this
***

### Environment Information
<table align='left'>
<tr>
    <th>Environment Type</th>
    <th>Anaconda Version</th>
    <th>Anaconda Build Channel</th>
    <th>Python Version</th>
</tr>
<tr>
    <td>Anaconda </td>
    <td>2021.11 </td>
    <td>py39_0 </td>
    <td>3.9.11 </td>
</tr>
</table>

## Links to Kaggle Datasource: 
***

#### _[Transactional Retail Dataset of Electronics Store](https://www.kaggle.com/datasets/muhammadshahrayar/transactional-retail-dataset-of-electronics-store)_

*  _[dirty_data.csv](https://www.kaggle.com/datasets/muhammadshahrayar/transactional-retail-dataset-of-electronics-store?select=dirty_data.csv)_
*  _[missing_data.csv](https://www.kaggle.com/datasets/muhammadshahrayar/transactional-retail-dataset-of-electronics-store?select=missing_data.csv)_
*  _[warehouses.csv](https://www.kaggle.com/datasets/muhammadshahrayar/transactional-retail-dataset-of-electronics-store?select=warehouses.csv)_


# Table of Contents
- [3: Setup](#setup)
- [4: CSV Import and review](#CSVImport)
- [5: Detect and fix errors in df_dirty_data DataFrame](#Fix_df_dirty_data)
    - [Repair 'nearest_warehouse' name](#rename_nearest_warehouse)
    - [Investigate incorrect 'order_total' and 'order_price'](#order_price_total)
    - [Repair 'season' names](#rename_season)
    - [Repair 'customer_lat'itude and longitude](#fix_latitude)
- [6: Detect and fix errors in df_missing_data DataFrame](#Fix_df_missing_data)
    - [Identify and insert missing 'nearest_warehouse' values](#missing_nearest_warehouse)
    - [Identify and insert missing 'order_price' values](#missing_order_price)
    - [Identify and insert missing 'order_total' values](#missing_order_total)
    - [Identify and insert missing 'season' values](#missing_season)
    - [Identify and insert missing 'distance_to_nearest_warehouse' values](#missing_distance_to_nearest_warehouse)
- [7: Append cleaned Dataframes df_dirty_data and df_missing_data](#Appenddf_dirty_datadf_missing_data)
- [8: Create new df_customer_addresses DataFrame and export to CSV](#df_customer_addresses) 
- [9: Create new df_warehouse_addresses DataFrame and export to CSV](#df_warehouse_addresses) 
- [10: Merge df_order_data, df_customer_addresses and df_warehouse_addresses](#merge_df_order_data) 
- [11: Summary Statistics](#summary_statistics) 
- [12: Visualisations](#Visualisations)





<a id='setup'></a>

# Setup

<div class='alert alert-block alert-info'><b></b>installing an importing modules, setting variables and functions </div>

## Install Modules

In [ ]:
# modules required
%pip install matplotlib
%pip install pandas
%pip install seaborn
%pip install geopy
%pip install plotly

## Import Modules
***

In [ ]:
from matplotlib import pyplot as plt

import pandas as pd
import numpy as np
import seaborn as sns

import time
from datetime import datetime

from geopy.distance import geodesic
from geopy.geocoders import Nominatim
import plotly.express as px

from IPython.display import display

## Set Variables
***

In [ ]:
# set all csv variables
dirty_data = 'dirty_data.csv'
missing_data = 'missing_data.csv'
warehouse_data = 'warehouses.csv'
customer_addresses = 'customer_addresses.csv' 
warehouse_addresses = 'warehouse_addresses.csv'

create_df_customer_addresses = 0 # set to 1 if you wnat section 8 to run:
create_df_warehouse_addresses = 0 # set to 1 if you wnat section 9 to run:

## Functions
***

In [ ]:
def get_nearest_warehouse(lat, Long ):
    ''' use coordinates to find the closest warehouse (crow flies) using geodesic API'''
    
    # test values:  get_nearest_warehouse(-37.815105, 144.932843)
    
    # customer coordinates are provided
    coords_customer = lat, Long
    
    # get warehouse latitude and longitide from warehouse table
    coords_Nickolson = df_warehouse_data['lat'].loc[df_warehouse_data.index[0]], df_warehouse_data['lon'].loc[df_warehouse_data.index[0]]
    coords_Thompson = df_warehouse_data['lat'].loc[df_warehouse_data.index[1]], df_warehouse_data['lon'].loc[df_warehouse_data.index[1]]
    coords_Bakers = df_warehouse_data['lat'].loc[df_warehouse_data.index[2]], df_warehouse_data['lon'].loc[df_warehouse_data.index[2]]
    
    # use geodesic to do a km distance comparison
    Nickolson_cust_dist = geodesic(coords_customer, coords_Nickolson).km
    Thompson_cust_dist = geodesic(coords_customer, coords_Thompson).km
    Bakers_cust_dist = geodesic(coords_customer, coords_Bakers).km
    
    val = (Nickolson_cust_dist, Thompson_cust_dist, Bakers_cust_dist)
    
    # get the min value distance value
    min_dist = val.index(min(val))
    
     # convert min_dist to warehouse name and distance in km
    if min_dist == 0:
        warehouse = 'Nickolson'
        distance = Nickolson_cust_dist
        return(warehouse, distance)
    elif min_dist == 1:
        warehouse = 'Thompson'
        distance = Nickolson_cust_dist
        return(warehouse, distance)
    else: 
        warehouse = 'Bakers'
        distance = Nickolson_cust_dist
        
        return str(warehouse), distance        

In [ ]:
def get_order_price(order_total, delivery_charges, coupon_discount):
    ''' calculate order_price from order_total, delivery_charges and coupon_discount '''
        # test:  get_order_price(11059.89, 79.89, 10 )
        
    order_price = (order_total - delivery_charges) / (100-coupon_discount) * 100
    return round(order_price, 2) 

In [ ]:
def get_order_total(order_price, delivery_charges, coupon_discount):
    ''' calculate order_total from order_price, delivery_charges and coupon_discount'''
    # test values : get_order_total(11059.89, 79.89, 10 )
    
    order_total = order_price - (order_price / 100 * coupon_discount) + delivery_charges
    return round(order_total, 2)


In [ ]:
def get_season(date):
    '''calculate season from date'''
    # test values : get_season(datetime(2019, 6, 22))
    
    # Spring = months 9,10,11
    # summer = months 12,1,2
    # Autumn = months 3,4,5
    # Winter = months 6,7,8

    month = date.month

    if month in [9,10,11]:
        season = 'Spring'
    elif month in [12,1,2]:
        season = 'Summer'
    elif month in [3,4,5]:
        season = 'Autumn'
    else:
        season = 'Winter'
    
    return season

In [ ]:
def get_geo_address(customer_id, lat, long):
    '''use  Nominatim API to get address data from customer_lat and customer_long'''
    # test values:  get_geo_address('ID6231506320', -37.8117454, 144.9564037)
    
    # create an instance     
    geolocator = Nominatim(user_agent='app')
      
    coordinates =  str(lat) + ' ' + str(long)
    
    #apply the method to get the location
    location = geolocator.reverse(coordinates,timeout=None)
    # return the required address      
    address = location.raw['address']
    road = address.get('road', '')
    suburb = address.get('suburb', '')
    city = address.get('city', '')
    postcode = address.get('postcode', '')
    state = address.get('state', '')
    country = address.get('country', '')
    
    return customer_id, road, suburb, city, postcode, state, country

In [ ]:
def convert_to_negative(val):
    '''get the negative of a passed value'''
    # test values:  convert_to_negative(37.8117454)
    
    # simple multiply by -1     
    negative_val = val * (-1)
    
    return negative_val

<a id='CSVImport'></a>

# CSV Import and review
***

<div class='alert alert-block alert-info'><b></b>Create DataFrames<strong>  df_dirty_data</strong>, <strong>df_missing_data</strong>, <strong>df_warehouse_data</strong>, <strong>df_customer_addresses</strong> and <strong>df_warehouse_addresses</strong><br>Imported from relevent CSVs<br>Datatype changes are made on import where possible, some indexes set and columns selected</div>

In [ ]:
# read the CSV from the local Jupyter Notebook directory 
# parsing dates on import since dates were objects also change datatype on order_price
df_dirty_data = pd.read_csv(dirty_data, parse_dates=['date'], dtype={'order_price': 'float'}) 
df_missing_data = pd.read_csv(missing_data, parse_dates=['date']) 
df_warehouse_data = pd.read_csv(warehouse_data)
df_customer_addresses = pd.read_csv(customer_addresses, dtype={'postcode': 'int'}, usecols=['customer_id','road','suburb','city','postcode','state','country'],index_col='customer_id') #this csv is created for this assignment in Section 7
df_warehouse_addresses = pd.read_csv(warehouse_addresses, dtype={'postcode': 'int'}, usecols=['warehouse','road','suburb','city','postcode','state','country'], index_col='warehouse')  #this csv is created for this assignment in Section 8

## Data Review
***

### df_dirty_data dataframe review

In [ ]:
type(df_dirty_data)

In [ ]:
print(df_dirty_data.columns)

In [ ]:
# will use display command for better tabular formatting in jupyter Notebook
display(df_dirty_data.head())

In [ ]:
# will use display command for better tabular formatting in jupyter Notebook
display(df_dirty_data.describe())

In [ ]:
print(df_dirty_data.info())

In [ ]:
print(df_dirty_data.shape)

In [ ]:
# no NaN found
print(df_dirty_data.isnull().any())

In [ ]:
missing_values_count = df_dirty_data.isnull().sum()
print(missing_values_count[0:16])

### df_missing_data dataframe review

In [ ]:
type(df_missing_data)

In [ ]:
print(df_missing_data.columns)

In [ ]:
display(df_missing_data.head())

In [ ]:
display(df_missing_data.describe())

In [ ]:
print(df_missing_data.info())

In [ ]:
print(df_missing_data.shape)

In [ ]:
print(df_missing_data.isnull().any())

In [ ]:
print(df_missing_data.isnull().sum())

### df_warehouse_data dataframe review

In [ ]:
type(df_warehouse_data)

In [ ]:
print(df_warehouse_data.columns)

In [ ]:
type(df_warehouse_data)

In [ ]:
display(df_warehouse_data.head())

In [ ]:
display(df_warehouse_data.describe())

In [ ]:
print(df_warehouse_data.info())

In [ ]:
print(df_warehouse_data.shape)

In [ ]:
print(df_warehouse_data.isnull().any())

In [ ]:
print(df_warehouse_data.isnull().sum())

### df_customer_addresses dataframe review

[This DataFrame is created in Section 8](#df_customer_addresses)

In [ ]:
type(df_customer_addresses)

In [ ]:
print(df_customer_addresses.columns)

In [ ]:
type(df_customer_addresses)

In [ ]:
display(df_customer_addresses.head())

In [ ]:
display(df_customer_addresses.describe())

In [ ]:
print(df_customer_addresses.info())

In [ ]:
print(df_customer_addresses.shape)

In [ ]:
print(df_customer_addresses.isnull().any())

In [ ]:
print(df_customer_addresses.isnull().sum())

### df_warehouse_addresses dataframe review

[This DataFrame is created in Section 9](#df_warehouse_addresses)

In [ ]:
type(df_warehouse_addresses)

In [ ]:
print(df_warehouse_addresses.columns)

In [ ]:
type(df_warehouse_addresses)

In [ ]:
display(df_warehouse_addresses.head())

In [ ]:
display(df_warehouse_addresses.describe())

In [ ]:
print(df_warehouse_addresses.info())

In [ ]:
print(df_warehouse_addresses.shape)

In [ ]:
print(df_warehouse_addresses.isnull().any())

In [ ]:
print(df_warehouse_addresses.isnull().sum())

## Create Indexes

In [ ]:
# check for orderid uniqueness in df_dirty_data
duplicate_dirty_data = df_dirty_data[df_dirty_data.duplicated(['order_id'])]
print(duplicate_dirty_data)

In [ ]:
# check for orderid uniqueness in df_missing_data 
duplicate_missing_data = df_missing_data[df_missing_data.duplicated(['order_id'])]
print(duplicate_missing_data)

In [ ]:
# create an order_id index on df_dirty_data
df_dirty_data.set_index('order_id',inplace=True)

In [ ]:
# create an order_id index on df_missing_data
df_missing_data.set_index('order_id',inplace=True)

In [ ]:
print(df_dirty_data.index)

In [ ]:
print(df_missing_data.index)

<a id='Fix_df_dirty_data'></a>

# Detect and fix errors in df_dirty_data DataFrame
***

<div class='alert alert-block alert-info'><b></b>Fixing anomolous data issues with <strong>df_dirty_data</strong> DataFrame</div>

In [ ]:
# show missing values - none found
print(df_dirty_data.isnull().sum())

In [ ]:
print(df_dirty_data.info())

In [ ]:
df_dirty_data['customer_id'].sort_values().unique()

In [ ]:
df_dirty_data['customer_id'].apply(len).unique()

In [ ]:
type(df_dirty_data['customer_id'].sort_values().unique())

In [ ]:
display(df_dirty_data.head())

In [ ]:
columns = df_dirty_data.columns
print(len(columns))

<a id='rename_nearest_warehouse'></a>

### Repair 'nearest_warehouse name

In [ ]:
# getting the unique values for 'nearest_warehouse' 
nearest_warehouse = df_dirty_data['nearest_warehouse'].sort_values().unique()
print(nearest_warehouse)

In [ ]:
# capatilise the first character 
df_dirty_data['nearest_warehouse'] = df_dirty_data['nearest_warehouse'].str.title()

<a id='rename_season'></a>

### Repair 'season' names

In [ ]:
# getting the unique values for 'season'
season = df_dirty_data['season'].sort_values().unique()
print(season)

In [ ]:
# capatilise the first character 
df_dirty_data['season'] = df_dirty_data['season'].str.title()

<a id='order_price_total'></a>

### Repair incorrect 'order_total' and 'order_price'

In [ ]:
price = df_dirty_data['order_price']

print('Q2 quantile of order_price : ', np.quantile(price, .50))
print('Q1 quantile of order_price : ', np.quantile(price, .25))
print('Q3 quantile of order_price : ', np.quantile(price, .75))
print('100th quantile of order_price : ', np.quantile(price, .1))

In [ ]:
total = df_dirty_data['order_total']

print('Q2 quantile of order_total : ', np.quantile(total, .50))
print('Q1 quantile of order_total : ', np.quantile(total, .25))
print('Q3 quantile of order_total : ', np.quantile(total, .75))
print('100th quantile of order_total : ', np.quantile(total, .1))

### visually comparing 'shopping_cart'  'order_price'  'delivery_charges'  'coupon_discount'  'order_total' 

In [ ]:
# sorted by 'order_total','order_price'
df_dirty_data[['shopping_cart','order_price','delivery_charges','coupon_discount','order_total']].sort_values(ascending = False, by=['order_total','order_price']).head(10)

In [ ]:
# sorted by 'order_price','order_total'
df_dirty_data[['shopping_cart','order_price','delivery_charges','coupon_discount','order_total']].sort_values(ascending = False, by=['order_price','order_total']).head(10)

In [ ]:
# sorted by 'order_price','shopping_cart' perhaps anomolies are from items purchased 
df_dirty_data[['shopping_cart','order_price','delivery_charges','coupon_discount','order_total']].sort_values(ascending = False, by=['order_price','shopping_cart']).head(10)

In [ ]:
pd.set_option('display.max_colwidth', 100)
df_dirty_data[['shopping_cart','order_price','delivery_charges','coupon_discount','order_total']].sort_values(ascending = True, by=['order_price']).head()

In [ ]:
pd.set_option('max_colwidth', 100)
df_shopping_cart = df_dirty_data[['shopping_cart','order_price']]

# df_shopping_cart_filter = df_shopping_cart[lambda row : row['shopping_cart'].str.contains('Thunder line.*Lucent 330S', regex=True)]
df_shopping_cart_filter = df_shopping_cart[lambda row : row['shopping_cart'].str.contains('Thunder line.*Lucent 330S|Lucent 330S.*Thunder line', regex=True)].head()

df_shopping_cart_filter.sort_values('shopping_cart')
# unique items 'Thunder line', 'Lucent 330S', 'Alcon 10', 'Universe Note', 'Olivia x460', 'Candle Inferno', 'iAssist Line', 'iStream', 'Toshika 750', 'pearTV'

In [ ]:
# if we look at order_price versus order_total there are some outliers 
sns.scatterplot(data=df_dirty_data,y='order_total', x='order_price');

In [ ]:
# Minimum and maximum order_price versus order_total 
print('Minimum order_price ' + str(df_dirty_data['order_price'].min()));
print('Minimum order_total ' + str(df_dirty_data['order_total'].min()));
print('Maximum order_price ' + str(df_dirty_data['order_price'].max()));
print('Maximum order_total ' + str(df_dirty_data['order_total'].max()));

In [ ]:
df_dirty_data.head()

#### calculating the actual order_total that dont add up from 'order_price' 'delivery_charges' 'coupon_discount', listing those order_ids 

In [ ]:
# lets find where (order_price - coupon_discount + delivery_charges != order_total) dont add up 
# we can use function get_order_total to get order_id where they dont match 
# add incorrect rows to new dataframe df_incorrect_order_total

df_incorrect_order_total = pd.DataFrame()

for i, row in df_dirty_data.iterrows():
    if df_dirty_data.at[i, 'order_total'] != get_order_total(df_dirty_data.at[i,'order_price'],df_dirty_data.at[i,'delivery_charges'],df_dirty_data.at[i,'coupon_discount']):
        df_incorrect_order_total = pd.concat([df_incorrect_order_total, pd.DataFrame.from_records([{'order_id':i, 'order_price':df_dirty_data.at[i,'order_price'],
                                                                                                    'delivery_charges':df_dirty_data.at[i,'delivery_charges'],
                                                                                                    'coupon_discount':df_dirty_data.at[i,'coupon_discount'],
                                                                                                    'order_total':df_dirty_data.at[i,'order_total']}])],ignore_index=True)

In [ ]:
display(df_incorrect_order_total.sort_values('order_total', ascending=False))

#### calculating the actual order_price that dont add up from 'order_total' 'delivery_charges' 'coupon_discount', listing those order_ids

In [ ]:
# lets find where (order_total - delivery_charges / coupon_discount != order_price) dont add up
# we can use function get_order_price to get order_id where they dont match
# add incorrect rows dataframe df_incorrect_order_price

df_incorrect_order_price = pd.DataFrame()

for i, row in df_dirty_data.iterrows():
    if df_dirty_data.at[i,'order_price'] != get_order_price(df_dirty_data.at[i,'order_total'],df_dirty_data.at[i,'delivery_charges'],df_dirty_data.at[i,'coupon_discount']):
        df_incorrect_order_price = pd.concat([df_incorrect_order_price, pd.DataFrame.from_records([{'order_id':i,                                               
                'order_price':df_dirty_data.at[i,'order_price'],                                 
                'delivery_charges':df_dirty_data.at[i,'delivery_charges'],
                'coupon_discount':df_dirty_data.at[i,'coupon_discount'], 
                'order_total':df_dirty_data.at[i,'order_total']}])],ignore_index=True)

In [ ]:
display(df_incorrect_order_price.sort_values('order_total', ascending=False))

In [ ]:
# we can conclude that 40 orders have incorrect calculations for either order_price or order_total,they will be removed 
# it makes sense that df_incorrect_order_price and df_incorrect_order_total contain the same rows 
df_incorrect_order_id = df_incorrect_order_price['order_id'] = df_incorrect_order_total['order_id']
print(df_incorrect_order_id.count())

In [ ]:
# list of order_id's that are incorrect
list_incorrect_order_id = df_incorrect_order_id.tolist()
print(list_incorrect_order_id)

### remove list_incorrect_order_id from df_dirty_data 

In [ ]:
# use the list created above 'list_incorrect_order_id' to remove orders that have incorrect order_price and order_total's 
print(df_dirty_data.drop(df_dirty_data[df_dirty_data.index.isin(list_incorrect_order_id)].index,inplace=True))

In [ ]:
df_dirty_data.head()

<a id='fix_latitude'></a>

### Repair 'customer_lat'itude and longitude

#### Check Latitude and Longitude in df_dirty_data

In [ ]:
max_long = df_dirty_data['customer_long'].max();
min_long = df_dirty_data['customer_long'].min()
max_lat = df_dirty_data['customer_lat'].max();
min_lat = df_dirty_data['customer_lat'].min()
print('max_longitude:',max_long)
print('max_latitude:',max_lat)
print('min_longitude:',min_long)
print('min_latitude:',min_lat)

In [ ]:
# getting index where 'customer_lat' is positive
positive_customer_lat_indexto_update = df_dirty_data[df_dirty_data['customer_lat'] > 0 ].index
print(positive_customer_lat_indexto_update)

In [ ]:
#  using a scatterplot to visualise discrepancies in Latitude and Longitude in the df_dirty_data DataFrame
#  the scatterplot shows there are some positive latitude values, we'll map these to see where they are
plt.figure(figsize = (10,6))
sns.set_theme(style='darkgrid');

ax = sns.scatterplot(x='customer_lat', y='customer_long', data=df_dirty_data, hue='customer_lat', palette = 'vlag')
plt.title('Customer Coordinates')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.show()  

In [ ]:
# map shows coordinates are in the Melbourne area and off the Japanese coast, unless Captain Nemo ordered some supplies im assuming this to be incorrect
# zoom on the map for closer inspection
# needs WebGL enabled in browser
fig = px.scatter_mapbox(df_dirty_data, lat='customer_lat', lon='customer_long',color_discrete_sequence=['DarkRed'], zoom=2, height=600, center=dict(lat=-4, lon=160), size='customer_long')
fig.update_layout(mapbox_style='open-street-map',margin={'r':500,'t':0,'l':0,'b':0})
fig.show()

In [ ]:
# df_dirty_data.describe shows the max value
display(df_dirty_data.describe()[['customer_lat']])
# min and percentiles are in the -37 decimal range with the exception of the max 37, I suggest these latitudes should be negative, will change and observe the difference 
# if making the values negative re-positions to Melbourne it is probably ok

In [ ]:
# list all rows where 'customer_lat' is positive
positive_customer_lat_to_update = df_dirty_data[df_dirty_data['customer_lat'] > 0 ]
display(positive_customer_lat_to_update[['customer_lat']])

In [ ]:
# getting index where 'customer_lat' is positive
positive_customer_lat_indexto_update = df_dirty_data[df_dirty_data['customer_lat'] > 0 ].index
print(positive_customer_lat_indexto_update)

In [ ]:
# updating  values for 'customer_lat'
for i, row in positive_customer_lat_to_update.iterrows():
    df_dirty_data.at[i,'customer_lat'] = convert_to_negative(df_dirty_data.at[i,'customer_lat'])

In [ ]:
# list all rows where 'customer_lat' is positive : all fixed
positive_customer_lat_to_update = df_dirty_data[df_dirty_data['customer_lat'] > 0 ]
display(positive_customer_lat_to_update)

In [ ]:
#check the indexes for the updated 'customer_lat' --  all done
display(df_dirty_data[['customer_lat']].loc[positive_customer_lat_indexto_update])

In [ ]:
# using a scatterplot to visualise updated in Latitude and Longitude in the df_dirty_data DataFrame
# all values are negative and all within -37
plt.figure(figsize = (10,6))
sns.set_theme(style='darkgrid');

sns.scatterplot(x='customer_lat', y='customer_long', data=df_dirty_data, hue='customer_lat', palette = 'vlag')
plt.title('Customer Coordinates')
plt.xlabel('Latitude')
plt.ylabel('Longitude')
plt.show()

In [ ]:
# check the updated coordinates are in the Melbourne area, all seem to be good, all located downtown Melbourne and north of Melbourne
# zoom on the map for closer inspection
# needs WebGL enabled in browser
fig = px.scatter_mapbox(df_dirty_data, lat='customer_lat', lon='customer_long',color_discrete_sequence=['DarkRed'],center=dict(lat=-37.5, lon=145), zoom=7)
fig.update_layout(mapbox_style='open-street-map',margin={'r':500,'t':0,'l':0,'b':0})
fig.show()

#### Check Latitude and Longitude in df_missing_data

In [ ]:
max_long = df_missing_data['customer_long'].max();
min_long = df_missing_data['customer_long'].min()
max_lat = df_missing_data['customer_lat'].max();
min_lat = df_missing_data['customer_lat'].min()
print('max_longitude:',max_long)
print('max_latitude:',max_lat)
print('min_longitude:',min_long)
print('min_latitude:',min_lat)

In [ ]:
# here we can see the scatterplot contains an even distribution for negative latitude values in the df_missing_data DataFrame
# all -37
plt.figure(figsize = (10,6))
sns.set_theme(style='darkgrid');

sns.scatterplot(x='customer_lat', y='customer_long',data=df_missing_data,  hue='customer_lat', palette = 'vlag')
plt.title('Cutomer Coordinates',)
# Set x-axis label
plt.xlabel('Latitude')
# Set y-axis label
plt.ylabel('Longitude')
plt.show()

In [ ]:
# map shows all coordinates are in the Melbourne area in df_missing_data nothing seems out of order
# zoom on the map for closer inspection
fig = px.scatter_mapbox(df_missing_data, lat='customer_lat', lon='customer_long',color_discrete_sequence=['DarkRed'], zoom=11)
fig.update_layout(mapbox_style='open-street-map',margin={'r':500,'t':0,'l':0,'b':0})
fig.show()

<a id='Fix_df_missing_data'></a>

# Detect and Fix missing values in df_missing_data DataFrame
***

<div class='alert alert-block alert-info'><b></b>Fixing missing data issues with <strong>df_missing_data</strong> DataFrame</div>

In [ ]:
# show missing values
print(df_missing_data.isnull().sum())

<a id='missing_nearest_warehouse'></a>

### Identify and insert missing 'nearest_warehouse' values

In [ ]:
# getting rows for the missing values for 'nearest_warehouse'
nearest_warehouse_to_update = df_missing_data[df_missing_data['nearest_warehouse'].isnull()]

display(nearest_warehouse_to_update[['nearest_warehouse']])

In [ ]:
# getting index for the the missing values for 'nearest_warehouse'
nearest_warehouse_indexto_update = df_missing_data[df_missing_data['nearest_warehouse'].isnull()].index
print(nearest_warehouse_indexto_update)

In [ ]:
# updating missing values for 'nearest_warehouse'
for i, row in nearest_warehouse_to_update.iterrows():
    df_missing_data.at[i,'nearest_warehouse'] = get_nearest_warehouse(df_missing_data.at[i,'customer_lat'],df_missing_data.at[i,'customer_long'])[0]

In [ ]:
# no more missing values for 'nearest_warehouse'
print(df_missing_data[df_missing_data['nearest_warehouse'].isnull()])

In [ ]:
#check the indexes for the update 'nearest_warehouse' --  all done
display(df_missing_data[['nearest_warehouse']].loc[nearest_warehouse_indexto_update])

<a id='missing_order_price'></a>

### Identify and insert missing 'order_price' values

In [ ]:
# getting rows for the missing values for 'order_price'
order_price_to_update = df_missing_data[df_missing_data['order_price'].isnull()]

display(order_price_to_update[['order_price']])

In [ ]:
# getting index for the the missing values for 'order_price'
order_price_indexto_update = df_missing_data[df_missing_data['order_price'].isnull()].index
print(order_price_indexto_update)

In [ ]:
# updating missing values for 'order_price'
for i, row in order_price_to_update.iterrows():
    df_missing_data.at[i,'order_price'] = get_order_price(df_missing_data.at[i,'order_total'],df_missing_data.at[i,'delivery_charges'],df_missing_data.at[i,'coupon_discount'])

In [ ]:
# no more missing values for 'order_price'
print(df_missing_data[df_missing_data['order_price'].isnull()])

In [ ]:
#check the indexes for the update 'order_price' --  all done
display(df_missing_data[['order_price']].loc[order_price_indexto_update])

<a id='missing_order_total'></a>

### Identify and insert missing 'order_price' values

In [ ]:
# getting rows for the missing values for 'order_total'
order_total_to_update = df_missing_data[df_missing_data['order_total'].isnull()]

display(order_total_to_update[['order_total']])

In [ ]:
# getting index for the the missing values for 'order_total'
order_total_indexto_update = df_missing_data[df_missing_data['order_total'].isnull()].index
print(order_total_indexto_update)

In [ ]:
# updating missing values for 'order_total'
for i, row in order_total_to_update.iterrows():
    df_missing_data.at[i,'order_total'] = get_order_total(df_missing_data.at[i,'order_price'],df_missing_data.at[i,'delivery_charges'],df_missing_data.at[i,'coupon_discount'])

In [ ]:
# no more missing values for 'order_total'
print(df_missing_data[df_missing_data['order_total'].isnull()])

In [ ]:
#check the indexes for the update 'order_total' --  all done
display(df_missing_data[['order_total']].loc[order_total_indexto_update])

<a id='missing_season'></a>

### Identify and insert missing 'season' values

In [ ]:
# getting rows for the missing values for 'season'
season_to_update = df_missing_data[df_missing_data['season'].isnull()]

display(season_to_update[['season']])

In [ ]:
# getting index for the the missing values for 'order_total'
season_indexto_update = df_missing_data[df_missing_data['season'].isnull()].index
print(season_indexto_update)

In [ ]:
# getting the unique values for 'season'
seasons = df_missing_data['season'].sort_values().unique()
print(seasons)

In [ ]:
# get the season months from the grouping of the orders..obviously Southern hemisphere - a random lat long shows its Melbourne, Australia area
season_months = df_missing_data.groupby([pd.Grouper(key='date', axis=0, freq='M'),'season']).sum().filter(items=['date','season'])
display(season_months)
# summer = months 12,1,2
# Autumn = months 3,4,5
# Winter = months 6,7,8
# Spring = months 9,10,11

In [ ]:
# updating missing values for 'season'
for i, row in df_missing_data.iterrows():
    df_missing_data.at[i,'season'] = get_season(df_missing_data.at[i,'date'])

In [ ]:
# no more missing values for 'season'
print(df_missing_data[df_missing_data['season'].isnull()])

In [ ]:
#check the indexes for the update 'season' --  all done
display(df_missing_data[['season']].loc[order_total_indexto_update])

<a id='missing_distance_to_nearest_warehouse'></a>

### Identify and insert missing 'distance_to_nearest_warehouse' values

In [ ]:
# getting rows for the missing values for 'distance_to_nearest_warehouse'
distance_to_nearest_warehouse_to_update = df_missing_data[df_missing_data['distance_to_nearest_warehouse'].isnull()]

display(distance_to_nearest_warehouse_to_update[['distance_to_nearest_warehouse']])

In [ ]:
# getting index for the the missing values for 'distance_to_nearest_warehouse'
distance_to_nearest_warehouse_indexto_update = df_missing_data[df_missing_data['distance_to_nearest_warehouse'].isnull()].index
print(distance_to_nearest_warehouse_indexto_update)

In [ ]:
# updating missing values for 'distance_to_nearest_warehouse'
for i, row in distance_to_nearest_warehouse_to_update.iterrows():
    df_missing_data.at[i,'distance_to_nearest_warehouse'] = get_nearest_warehouse(df_missing_data.at[i,'customer_lat'],df_missing_data.at[i,'customer_long'])[1]

In [ ]:
# no more missing values for 'distance_to_nearest_warehouse'
print(df_missing_data[df_missing_data['distance_to_nearest_warehouse'].isnull()])

In [ ]:
#check the indexes for the update 'season' --  all done
display(df_missing_data[['distance_to_nearest_warehouse']].loc[order_total_indexto_update])

<a id='Appenddf_dirty_datadf_missing_data'></a>

# Append cleaned Dataframes df_dirty_data and df_missing_data
***

<div class='alert alert-block alert-info'><b></b>Work is complete on correcting issues with <strong>df_missing_data</strong> and <strong>df_dirty_data</strong> they are appended here to create <strong>df_order_data</strong><br> NaN are filtered from 'customer_lat', 'customer_long' and 'is_happy_customer'</div>

In [ ]:
df_missing_data.shape

In [ ]:
df_missing_data.info()

In [ ]:
df_missing_data.index

In [ ]:
df_missing_data.isnull().sum()

In [ ]:
df_dirty_data.shape

In [ ]:
df_dirty_data.info()

In [ ]:
df_dirty_data.index

In [ ]:
df_dirty_data.isnull().sum()

In [ ]:
# remove NaN for 'customer_lat', 'customer_long' and 'is_happy_customer' from df_missing_data
df_missing_data_NoNa = df_missing_data[(df_missing_data['customer_lat'].notna()) & (df_missing_data['customer_long'].notna()) & (df_missing_data['is_happy_customer'].notna())]
display(df_missing_data_NoNa)

In [ ]:
# create a new dataframe from df_dirty_data_NoNa and df_missing_data_NoNa 
# df_order_data = df_dirty_data_NoNa.append(df_missing_data_NoNa, ignore_index=False)
df_order_data  = pd.concat([df_dirty_data, df_missing_data_NoNa], verify_integrity=True)

# df_order_data = pd.concat([df_missing_data_NoNa, df_dirty_data_NoNa ], ignore_index=False)

In [ ]:
df_order_data = df_order_data.reset_index()

In [ ]:
display(df_order_data)

In [ ]:
df_order_data.order_id.unique

<a id='df_customer_addresses'></a>

# Create new df_customer_addresses DataFrame and export to CSV
***

<div class='alert alert-block alert-info'><b></b><strong>df_customer_addresses</strong> is created here from <strong>df_order_data</strong>.<br><strong>df_customer_addresses</strong> contains addresses for customers by making an Nominatum API call using the <strong>get_geo_address</strong> function<br><strong>df_customer_addresses</strong> is exported to <strong>customer_addresses.csv</strong> for re-use
<br><strong>create_df_customer_addresses = 0 </strong>so nothing will be created since i already created <strong>customer_addresses.csv</strong> for this assignment. <br>Set <strong>create_df_customer_addresses = 1 </strong> to recreate the dataset and csv</div>

In [ ]:
# getting 'customer_lat' and 'customer_long'] for df_order_data
customerid_lat_long = df_order_data[['customer_id','customer_lat','customer_long']]

display(customerid_lat_long.head(10))

In [ ]:
# get address for each customer and create a new Dataframe called df_customer_addresses
# create_df_customer_addresses = 0 so it doesnt run if 'All Cells are Run'
# Change create_df_customer_addresses = 1 to create the df_customer_addresses Dataframe, estimated time 970 records @ 1 second each ~ 17mins.. more like 30mins

if create_df_customer_addresses == 1:
    print('Creating df_customer_addresses Dataframe')
    df_customer_addresses = pd.DataFrame()
    x=1
    for i, row in customerid_lat_long.iterrows():
            #print to screen to get completion estimate from x     
            print(row[0],row[1],row[2])
            x+=1
            #add 1 second delay from Nominatum rate limiting         
            time.sleep(1)
            # pass values to the get_geo_address function          
            df_customer_addresses = df_customer_addresses.append({'customer_id':get_geo_address(row[0],row[1],row[2])[0], 
                        'road':get_geo_address(row[0],row[1],row[2])[1],
                        'suburb':get_geo_address(row[0],row[1],row[2])[2],
                        'city':get_geo_address(row[0],row[1],row[2])[3],
                        'postcode':get_geo_address(row[0],row[1],row[2])[4],                                 
                        'state':get_geo_address(row[0],row[1],row[2])[5], 
                        'country':get_geo_address(row[0],row[1],row[2])[6]
                       },ignore_index=True)
else:
    print('Dataframe df_warehouse_addresses is previously created from CSV, use this to create the CSV')

In [ ]:
duplicated = df_customer_addresses[df_customer_addresses.index.duplicated()]
display(duplicated)

In [ ]:
#  remove customer_address duplicates
df_customer_addresses = df_customer_addresses[~df_customer_addresses.index.duplicated(keep='first')] = df_customer_addresses[~df_customer_addresses.index.duplicated(keep='first')]

In [ ]:
df_customer_addresses

In [ ]:
# output df_customer_addresses created in previous step to csv in local notebook path
if create_df_customer_addresses == 1:
    df_customer_addresses.to_csv(r'customer_addresses.csv')

In [ ]:
display(df_customer_addresses)

<a id='df_warehouse_addresses'></a>

# Create new df_warehouse_addresses DataFrame and export to CSV
***

<div class='alert alert-block alert-info'><b></b><strong>df_warehouse_addresses</strong> is created here from <strong>df_warehouses</strong>.<br><strong>df_warehouse_addresses</strong> contains addresses for warehouses by making an Nominatum API call using the <strong>get_geo_address</strong> function<br><strong>df_warehouse_addresses</strong> is exported to <strong>warehouse_addresses.csv</strong> for re-use<br>
<strong>create_df_warehouse_addresses = 0 </strong>so nothing will be created since i already created <strong>warehouse_addresses.csv</strong> for this assignment. <br>Set <strong>create_df_warehouse_addresses = 1 </strong> to recreate the dataset and csv
</div>


In [ ]:
# getting '_lat' and 'customer_long'] for df_order_data
warehouse_lat_long = df_warehouse_data[['names','lat','lon']]

display(warehouse_lat_long)

In [ ]:
# get address for each customer and create a new Dataframe called df_customer_addresses
# Change create_df_warehouse_addresses = 1 to create the create_df_warehouse_addresses Dataframe, estimated time 970 records @ 1 second each ~ 17mins.. more like 30mins

if create_df_warehouse_addresses == 1:
    print('Creating df_warehouse_addresses Dataframe')
    df_warehouse_addresses = pd.DataFrame()
    x=1
    for i, row in warehouse_lat_long.iterrows():
            #print to screen to get completion estimate from x     
            print(x,row[0],row[1],row[2])
            x+=1
            #add 1 second delay from Nominatum rate limiting         
            time.sleep(1)
            # pass values to the get_geo_address function          
            df_warehouse_addresses = df_warehouse_addresses.append({'warehouse':get_geo_address(row[0],row[1],row[2])[0],
                        'road':get_geo_address(row[0],row[1],row[2])[1],
                        'suburb':get_geo_address(row[0],row[1],row[2])[2],
                        'city':get_geo_address(row[0],row[1],row[2])[3],
                        'postcode':get_geo_address(row[0],row[1],row[2])[4],                                 
                        'state':get_geo_address(row[0],row[1],row[2])[5], 
                        'country':get_geo_address(row[0],row[1],row[2])[6]
                       },ignore_index=True)
else:
    print('Dataframe df_warehouse_addresses not required')


In [ ]:
# output df_customer_addresses created in previous step to csv in local notebook path
if create_df_warehouse_addresses == 1:
    df_warehouse_addresses.to_csv(r'warehouse_addresses.csv')

In [ ]:
display(df_warehouse_addresses)

<a id='merge_df_order_data'></a>

# Merge df_order_data, df_customer_addresses and df_warehouse_addresses
***

<div class='alert alert-block alert-info'><b></b>Merging <strong>df_order_data</strong>, <strong>df_customer_addresses</strong> and <strong>df_warehouse_addresses</strong> to create <strong>df_order_data_full</strong></div>

In [ ]:
# data in df_order_data is cleanso we can merge with df_customer_addresses and df_warehouse_addresses
df_order_data_full = df_order_data.merge(df_customer_addresses, how='left', on=['customer_id'])\
.merge(df_warehouse_addresses, how='left',left_on='nearest_warehouse', right_on='warehouse', suffixes=('_cust','_warehouse'))

In [ ]:
df_order_data_full.shape

In [ ]:
df_order_data_full.isnull().sum()

In [ ]:
display(df_order_data_full[df_order_data_full['is_happy_customer']].head())

In [ ]:
display(df_order_data_full.columns)

<a id='summary_statistics'></a>

# Summary Statistics

In [ ]:
df_order_data_full['coupon_discount'].value_counts(sort=True)

In [ ]:
df_order_data_full['is_happy_customer'].value_counts(sort=True)

In [ ]:
df_order_data_full['nearest_warehouse'].value_counts(sort=True)

In [ ]:
df_order_data_full['nearest_warehouse'].value_counts(normalize=True)

In [ ]:
df_order_data_full.groupby('nearest_warehouse')['is_happy_customer'].mean()

In [ ]:
df_order_data_full.groupby('nearest_warehouse')['order_total'].agg([min, max, sum])

In [ ]:
df_order_data_full.pivot_table(values="order_total", index="nearest_warehouse", aggfunc=[np.mean, np.median])

In [ ]:
# function to sum calculations to compare to catplot

def SumWarehouseOrderTotal(warehouse):
    order_winter=df_order_data_full[np.logical_and(df_order_data_full['season'] == 'Winter', df_order_data_full['nearest_warehouse'] == warehouse)]['order_total'].sum()
    order_summer=df_order_data_full[np.logical_and(df_order_data_full['season'] == 'Summer', df_order_data_full['nearest_warehouse'] == warehouse)]['order_total'].sum()
    order_autumn=df_order_data_full[np.logical_and(df_order_data_full['season'] == 'Autumn', df_order_data_full['nearest_warehouse'] == warehouse)]['order_total'].sum()
    order_spring=df_order_data_full[np.logical_and(df_order_data_full['season'] == 'Spring', df_order_data_full['nearest_warehouse'] == warehouse)]['order_total'].sum()
    print(warehouse + ': Winter(sum): ',order_winter,'\n' + warehouse + ': Summer(sum): ',order_summer,'\n' + warehouse + ': Autumn(sum): ',order_autumn,'\n' + warehouse + ': Spring(sum): ',order_spring)

In [ ]:
SumWarehouseOrderTotal('Bakers')

In [ ]:
SumWarehouseOrderTotal('Thompson')

In [ ]:
SumWarehouseOrderTotal('Nickolson')

In [ ]:
warehouses = df_order_data_full.groupby(['nearest_warehouse'])['order_id'].count()
print(warehouses)

<a id='Visualisations'></a>

# Visualisations
***

<div class='alert alert-block alert-info'><b></b>Create Visualations for Analysis</div>

In [ ]:
df_order_data_full.columns

In [ ]:
#  total order price by happiness
sns.set_theme(style='darkgrid',palette='Set2')
plt.figure(figsize = (14, 10))

# Draw a sub violinplot and split for easier comparison
cp = sns.violinplot(data=df_order_data_full, x='nearest_warehouse', y='order_total', hue='is_happy_customer',split=True, inner='quart', linewidth=1)
cp.set_title('Order total per Warehouse by Customer Happiness ', fontsize=15);
cp.set_xlabel('\nWarehouse')
cp.set_ylabel('Order Total')
cp.legend(handles=ax.legend_.legendHandles, labels=['Unhappy', 'Happy'])
sns.despine(left=True)
# plt.savefig('Plot_Images/1_Total_Order_Price_by_Happiness.jpg')
plt.show();

In [ ]:
#  Delivery Charge by Happiness

sns.set_theme(style='darkgrid',palette='Set2')
plt.figure(figsize = (14, 10))

# Draw a sub violinplot and split for easier comparison
cp = sns.violinplot(data=df_order_data_full, x='nearest_warehouse', y='delivery_charges', hue='is_happy_customer',split=True, inner='quart', linewidth=1)
cp.set_title('Delivery Charge per Warehouse by Customer Happiness ', fontsize=15);
cp.set_xlabel('\nWarehouse')
cp.set_ylabel('Delivery Charge')
cp.legend(handles=ax.legend_.legendHandles, labels=['Unhappy', 'Happy'])
sns.despine(left=True)
# plt.savefig('Plot_Images/2_Delivery_Charge_by_Happiness.jpg')
plt.show();

In [ ]:
# calculate the order discount and add new column to df_order_data_full
df_order_data_full['order_discount'] = (df_order_data_full['order_price'] / 100) * df_order_data_full['coupon_discount']

In [ ]:
# Order Discount by Happiness

sns.set_theme(style='darkgrid',palette='Set2')
plt.figure(figsize = (14, 10))

# Draw a sub violinplot and split for easier comparison
cp = sns.violinplot(data=df_order_data_full, x='nearest_warehouse', y='order_discount', hue='is_happy_customer',split=True, inner='quart', linewidth=1)
cp.set_title('Order Discount per Warehouse by Customer Happiness ', fontsize=15);
cp.set_xlabel('\nWarehouse')
cp.set_ylabel('Order Discount')
cp.legend(handles=ax.legend_.legendHandles, labels=['Unhappy', 'Happy'])
sns.despine(left=True)
# plt.savefig('Plot_Images/3_Order_Discount_by_Happiness.jpg')
plt.show();

In [ ]:
# some order discounts seem high , checking if calculations are correct , all ok
df_order_data_full[df_order_data_full['order_discount'] > 5000].sort_values(by='order_discount', ascending=False).head()

In [ ]:
# some order discounts seem high , checking if calculations are correct , all ok
df_order_data_full[['order_price','coupon_discount']].sort_values(by=['order_price','coupon_discount'],ascending = False)

In [ ]:
# Order Totals by Warehouse
season_order_tot = df_order_data_full.groupby(['nearest_warehouse'])['order_total'].sum()

sns.set_theme(style='darkgrid',palette='Set2')
plt.figure(figsize = (14, 10))

season_order_tot.plot(kind = 'pie', title='Order Total by Warehouse')

fontdict = { 'fontsize': 30}
# plt.savefig('Plot_Images/4_Order_Total_by_Warehouse.jpg')
plt.show();

In [ ]:
# Total order value by Season grouped by Warehouse
sns.set_theme(style='darkgrid',palette='Set2')

cp = sns.catplot(x='nearest_warehouse', y='order_total', estimator=sum, data=df_order_data_full, kind='bar', hue='season',ci=None, legend=False)
cp.fig.set_size_inches(14, 10)
cp.fig.subplots_adjust(top=0.81,right=0.86) 
cp.set_ylabels('Sum of Order Total', fontsize=15)
cp.set_xlabels('Warehouse by Season', fontsize=15)
cp.set(title='Order Totals by Warehouse by Season')
cp.ax.legend(loc=1,fontsize=12)

ax = cp.facet_axis(0,0)
for pos in ax.patches:
    ax.text(pos.get_x() + .03, 
            pos.get_height() * 1.01, 
           '{0:.003f}M'.format(pos.get_height()/1000000),
            color='black', 
            rotation='horizontal', 
            size='medium')
# plt.savefig('Plot_Images/5_Total_Order_Value_by_Season_Grouped_by_Warehouse.jpg')
plt.show();

In [ ]:
# average distance to nearest Warehouse
sns.set_theme(style='darkgrid', palette='Set2')

cp = sns.catplot(x='nearest_warehouse', y='distance_to_nearest_warehouse', data=df_order_data_full, kind='bar', hue='season',ci=None, legend=False)
cp.fig.set_size_inches(14,10)
cp.fig.subplots_adjust(top=0.81,right=0.86) 
cp.set_ylabels('Distance to nearest Warehouse', fontsize=15)
cp.set_xlabels('Warehouse by Season', fontsize=15)
cp.set(title='Average Distance to Warehouse by Season')
cp.ax.legend(loc=1,fontsize=12)

ax = cp.facet_axis(0,0)
for pos in ax.patches:
    ax.text(pos.get_x() + .05, 
            pos.get_height() * 1.02, 
           '{0:.1f}'.format(pos.get_height()),
            color='black', 
            rotation='horizontal', 
            size='large')
# plt.savefig('Plot_Images/6_Average_Distance_to_Warehouse_by_Season.jpg')
plt.show();

In [ ]:
# Order Total by Season Over Time
sns.set_theme(style='darkgrid',palette='Set2')
plt.figure(figsize = (14, 10))
plt.title('Order Total by Season Over Time', fontsize=15)
sns.scatterplot(data=df_order_data_full, x='date', y='order_total', hue='season')
# plt.savefig('Plot_Images/7_Order_Total_by_Season_Over_Time.jpg')
plt.show();

In [ ]:
# Order Total by Warehouse Over Time
sns.set_theme(style='darkgrid',palette='Set2')
plt.figure(figsize = (14, 10))
plt.title('Order Total by Warehouse Over Time', fontsize=15)
sns.scatterplot(data=df_order_data_full, x='date', y='order_total', hue='nearest_warehouse')
# plt.savefig('Plot_Images/8_Order_Total_by_Warehouse_Over_Time.jpg')
plt.show();

In [ ]:
# Order Total by delivery charge Over Time'
plt.figure(figsize = (14, 10))
sns.set_theme(palette='Set2', style='darkgrid')
plt.title('Order Total by Delivery Charge Over Time', fontsize=15)
sns.scatterplot(data=df_order_data_full, x='order_total', y='delivery_charges', hue='delivery_charges')
# plt.savefig('Plot_Images/9_Order_Total_by_Delivery_Charge_Over_Time.jpg')
plt.show();

In [ ]:
# Total Daily Order Value versus Total Daily Delivery Charge
sns.set_theme(style='darkgrid',palette='Set2')

x =  df_order_data_full['date'].sort_values().unique()
data_1 = df_order_data_full.groupby(['date'])['order_total'].sum()
data_2 = df_order_data_full.groupby(['date'])['delivery_charges'].sum()

plt.figure(figsize=(14,8));

fig, ax1 = plt.subplots(figsize=(14,10))

ax1.set_xlabel('Date') 
ax1.set_ylabel('Total Daily Order Value') 
plot_1 = ax1.plot(x, data_1, label='order_total') 
ax1.tick_params(axis ='y');

ax2 = ax1.twinx() 
ax2.set_ylabel('Total daily Delivery Charge') 
plot_2 = ax2.plot(x, data_2,color='orange' ,label = 'delivery_charges') 
ax2.tick_params(axis ='y')

lns = plot_1 + plot_2 
labels = [l.get_label() for l in lns]
plt.title('Total Daily Order Value versus Total Daily Delivery Charge',fontsize=15)
plt.legend(lns, labels, loc=0)
plt.savefig('Plot_Images/10_Total_Daily_Order_Value_versus_Total_Daily_Delivery_Charge.jpg')
plt.show();